In [3]:
import os
from SignLanguageRecognition import signLanguageRecognizer


ModuleNotFoundError: No module named 'SignLanguageRecognition'

# Enfoque 1

In [15]:
def create_video_from_image(image_path, file_name):
    import cv2
    import os

    frame_rate = 31
    video_name = f"{file_name.split('.')[0]}.mp4"

    image_file = image_path + '/' + file_name
    frame = cv2.imread(os.path.join(image_file))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, 0, 1, (width,height))

    for frame in range(frame_rate):
        video.write(cv2.imread(os.path.join(image_file)))

    cv2.destroyAllWindows()
    video.release()

    return video_name

In [12]:
image_path = '../data/sign_language/'
arr = os.listdir(image_path)
for file_name in arr:
    video_name = create_video_from_image(image_path, file_name)
    signLanguageRecognizer.signLanguageRecognizerMethod(video_name)

# Enfqoque 2

In [2]:
import cv2
import os
import numpy as np
from keras.models import load_model

model = load_model('../models/leg_model.h5')
gestures = {
    1:'A',
    2:'B',
    3:'C',
    4:'D',
    5:'E',
    6:'F',
    7:'G',
    8:'H',
    9:'I',
    10:'K',
    11:'L',
    12:'M',
    13:'N',
    14:'O',
    15:'P',
    16:'Q',
    17:'R',
    18:'S',
    19:'T',
    20:'U',
    21:'V',
    22:'W',
    23:'X',
    24:'Y',
}


def predict(gesture):
    img = cv2.resize(gesture, (50,50))
    img = img.reshape(1,50,50,1)
    img = img/255.0
    prd = model.predict(img)
    index = prd.argmax()
    return gestures[index]

### Test 1

In [ ]:
image_path = '../data/sign_language/'
arr = os.listdir(image_path)
result_test1 = {}
for file_name in arr:
    gesture = cv2.imread(os.path.join(file_name))
    gesture = cv2.cvtColor(gesture, cv2.COLOR_BGR2GRAY)
    prediction = predict(gesture)
    result_test1[file_name] = prediction

### Test 2

In [ ]:
image_path = '../data/sign_language/'
arr = os.listdir(image_path)
result_test2 = {}
for file_name in arr:
    frame = cv2.imread(os.path.join(file_name))
    cv2.rectangle(frame, (100,100), (300,300), (0,255,0), 2)
    crop_img = frame[100:300, 100:300]
    blur = cv2.GaussianBlur(crop_img,(7,7),0)
    grey = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((5,5))
    dilation = cv2.dilate(grey,kernel,iterations=1)
    erosion = cv2.erode(dilation,kernel,iterations=1)
    blur1 = cv2.GaussianBlur(erosion,(5,5),0)
    thresh = cv2.threshold(blur1,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
    pred_text = predict(thresh)
    result_test2[file_name] = pred_text